In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE, trustworthiness
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from matplotlib.patches import Patch

In [ ]:
# Fill these up with your own values

EXCEL_PATH = "data.xlsx"
LABEL_COL  = "LABEL"

In [ ]:
# CONSTANTS

EPS = 1e-12                 # avoid taking log2(0)
N_BINS = 20                 # for discretisizing features

In [ ]:
# HELPERS

def kl_divergence(p, q, eps=EPS):
    """
    D_KL(p || q)
    """
    union_idx = p.index.union(q.index)
    p, q = p.reindex(union_idx, fill_value=0), q.reindex(union_idx, fill_value=0)
    return float(np.sum(p * np.log2((p + eps) / (q + eps))))

def get_distribution(series):
    """
    Get the PMF of a feature
    """
    return series.value_counts(normalize=True)

def discretise(series):
    """
    PMF might be near continuous so get_distribution would create 1/N tiny spikes in the dist.
    Seperate into bins
    """    
    if series.nunique() > N_BINS*2:
            return pd.qcut(series, q=N_BINS, duplicates="drop")
    return series

def shannon_entropy(p, eps=EPS):
    """
    H(p) (base-2)
    """
    return float(-np.sum(p * np.log2(p + eps)))

def js_divergence(p, q, eps=EPS):
    """
    Jensen-Shannon divergence.
    """
    union = p.index.union(q.index)
    p, q = p.reindex(union, fill_value=0), q.reindex(union, fill_value=0)
    m = 0.5 * (p + q)
    kl_p_m = np.sum(p * np.log2((p + eps) / (m + eps)))
    kl_q_m = np.sum(q * np.log2((q + eps) / (m + eps)))
    return 0.5 * (kl_p_m + kl_q_m)


In [ ]:
df        = pd.read_excel(EXCEL_PATH)
features  = [c for c in df.columns if c != LABEL_COL]

initial_rows = len(df)
df = df.dropna(axis=0)
final_rows = len(df)
print(f"Removed {initial_rows - final_rows} rows due to missing values.")
print(f"Remaining rows: {final_rows}")

final_class_counts = df[LABEL_COL].value_counts().sort_index()

for label, count in final_class_counts.items():
    print(f"  Class {label}: {count} patients")

In [ ]:
# COMPUTE THE UNCERTAINTY MATRIX

# Distributions, entropies (per class) and JS divergence (per feature)

dist      = {}
entropy   = {}
js        = {}

for feat in features:
    col_disc = discretise(df[feat])
    p1 = get_distribution(col_disc[df[LABEL_COL] == 1])
    p2 = get_distribution(col_disc[df[LABEL_COL] == 2])

    dist[feat]    = (p1, p2)
    entropy[feat] = {1: shannon_entropy(p1),
                     2: shannon_entropy(p2)}
    js[feat]      = max(js_divergence(p1, p2), EPS)    # never let it be 0

# Means & stds for z-scores
stats = {feat: {
             1: {"mu": df.loc[df[LABEL_COL] == 1, feat].mean(),
                 "std": df.loc[df[LABEL_COL] == 1, feat].std(ddof=0) + EPS},
             2: {"mu": df.loc[df[LABEL_COL] == 2, feat].mean(),
                 "std": df.loc[df[LABEL_COL] == 2, feat].std(ddof=0) + EPS}}
         for feat in features}


# Build the matrix
rows = []
for _, patient in df.iterrows():
    vec = []
    for feat in features:
        v = patient[feat]

        # z-scores vs each class
        z1 = (v - stats[feat][1]["mu"]) / stats[feat][1]["std"]
        z2 = (v - stats[feat][2]["mu"]) / stats[feat][2]["std"]

        # pick the magnitude-wise smaller one
        if abs(z1) < abs(z2):
            z  = z1
            cls = 1
        else:
            z  = z2
            cls = 2

        h      = entropy[feat][cls]       # Shannon entropy of the chosen class
        js_f   = js[feat]                 # JS divergence for this feature
        x_f    = z * (1.0 / (js_f + EPS)) * h

        vec.append(x_f)
    rows.append(vec)

X_matrix = pd.DataFrame(rows, index=df.index, columns=features)

In [ ]:
# sanity check

print("NaNs per feature:")
display(X_matrix.isna().sum().sort_values(ascending=False).head())

print("\nPatients with any NaN:")
na_rows = X_matrix.isna().any(axis=1).sum()
print(f"{na_rows} / {len(X_matrix)} patients contain at least one NaN")

In [ ]:
# scale

X_scaled = StandardScaler().fit_transform(X_matrix)

X_scaled.shape

In [ ]:
# do TSNE

tsne = TSNE(n_components=2, perplexity=50, learning_rate='auto',
            init='pca', random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

labels = df[LABEL_COL]  # 1 or 2

plt.figure(figsize=(7,6))

# Plot each class separately with label for legend
for cls, color in zip([1, 2], ['red', 'blue']):
    idx = labels == cls
    if cls == 1:
        plt.scatter(X_tsne[idx, 0], X_tsne[idx, 1],
                c=color, label=f'Myocarditis', s=35, alpha=0.7)
    if cls == 2:
        plt.scatter(X_tsne[idx, 0], X_tsne[idx, 1],
                c=color, label=f'ACS', s=35, alpha=0.7)


plt.xlabel('t-SNE-1')
plt.ylabel('t-SNE-2')

plt.gca().set_axis_off()

plt.legend(title="Class", loc='best')
plt.tight_layout()

plt.savefig("tSNE.pdf", dpi=300)
plt.show()

In [ ]:
X_tsne.shape

In [ ]:
# VISUALIZE

plt.rcParams["figure.figsize"] = (7, 6)
plt.rcParams["axes.titlesize"] = 13
plt.rcParams["axes.labelsize"] = 12

# Padding around extreme points so contours don't touch the frame
pad = 2.0

xmin, xmax = X_tsne[:, 0].min() - pad, X_tsne[:, 0].max() + pad
ymin, ymax = X_tsne[:, 1].min() - pad, X_tsne[:, 1].max() + pad

# 1000x1000 grid
resolution = 1000                 
xs = np.linspace(xmin, xmax, resolution)
ys = np.linspace(ymin, ymax, resolution)
xx, yy = np.meshgrid(xs, ys)
grid = np.vstack([xx.ravel(), yy.ravel()])


class1 = X_tsne[labels == 1]
class2 = X_tsne[labels == 2]

kde1 = gaussian_kde(class1.T, bw_method="scott")
kde2 = gaussian_kde(class2.T, bw_method="scott")

z1 = kde1(grid).reshape(xx.shape)
z2 = kde2(grid).reshape(xx.shape)

# 60 % iso-density
q = 0.6
level1 = np.quantile(z1, q)
level2 = np.quantile(z2, q)

overlap = (z1 >= level1) & (z2 >= level2)

def normalise(z, clip=0.98):
    """Scale density field to 0-1, clipping the top [clip] quantile."""
    zmax = np.quantile(z, clip)
    return np.clip(z / zmax, 0, 1)


In [ ]:
# Alpha masks for the pure class regions
alpha1 = normalise(z1)**0.5
alpha2 = normalise(z2)**0.5
alpha1[z1 < level1] = 0.0
alpha2[z2 < level2] = 0.0

# Identify the overlap and give it its own alpha map
overlap_mask      = (alpha1 > 0) & (alpha2 > 0)
alpha_overlap     = np.maximum(alpha1, alpha2)
alpha_overlap[~overlap_mask] = 0.0

# Remove the red/blue alphas inside the overlap so they don't sit on top
alpha1[overlap_mask] = 0.0
alpha2[overlap_mask] = 0.0

# Per-pixel colour for the overlap
eps   = 1e-12
total = z1 + z2 + eps
t     = (z1 - z2) / total
# -1 ⟶ pure blue side, +1 ⟶ pure red side

shift_strength = 0.3

# Base: all grey
R = np.full_like(t, 0.5)
G = np.full_like(t, 0.5)
B = np.full_like(t, 0.5)

# Shift toward red where t > 0
pos          = t > 0
R[pos] += shift_strength * t[pos]
G[pos] -= shift_strength * t[pos]
B[pos] -= shift_strength * t[pos]

# Shift toward blue where t < 0
neg          = t < 0
B[neg] += shift_strength * (-t[neg])
R[neg] -= shift_strength * (-t[neg])
G[neg] -= shift_strength * (-t[neg])

shape = (*alpha1.shape, 4)

red_img  = np.zeros(shape)
red_img[..., 0] = 1.0            
red_img[..., 3] = alpha1          

blue_img = np.zeros(shape)
blue_img[..., 2] = 1.0
blue_img[..., 3] = alpha2         

# Build the RGBA image for the overlap
shape      = (*alpha_overlap.shape, 4)
over_img   = np.zeros(shape)
over_img[..., 0] = R
over_img[..., 1] = G
over_img[..., 2] = B
over_img[..., 3] = alpha_overlap        # density-weighted fade-out


In [ ]:
# Plot
fig, ax = plt.subplots()

for img in [over_img, red_img, blue_img]:
    ax.imshow(img, extent=(xmin, xmax, ymin, ymax), origin="lower", interpolation="bilinear")

ax.set_axis_off()

legend_handles = [
    Patch(facecolor="red",  edgecolor="red",  label="Myocarditis"),
    Patch(facecolor="blue", edgecolor="blue", label="ACS"),
    Patch(facecolor="gray", edgecolor="gray", label="Uncertain")
]

ax.legend(handles=legend_handles, loc="upper right", frameon=False)

plt.tight_layout()
#plt.savefig("viz.pdf", dpi=300)
plt.show()